# Typewriter: Single Tool

Let's see how to evaluate an agent's ability to use tools.

    A task where the agent must type a given string one letter at a time.

    In this variation of the task, the agent is given a single function,
    that takes a letter as an argument.

In [1]:
from langchain_benchmarks import clone_public_dataset, registry

In [2]:
task = registry["Tool Usage - Typewriter (1 tool)"]
task

Name,Tool Usage - Typewriter (1 tool)
Type,ToolUsageTask
Dataset ID,59577193-8938-4ccf-92a7-e8a96bcf4f86
Description,"Environment with a single tool that accepts a single letter as input, and prints it on a piece of virtual paper. The objective of this task is to evaluate the ability of the model to use the provided tools to repeat a given input string. For example, if the string is 'abc', the tools 'a', 'b', and 'c' must be invoked in that order. The dataset includes examples of varying difficulty. The difficulty is measured by the length of the string."


Clone the dataset associaetd with this task

In [3]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Dataset Tool Usage - Typewriter (1 tool) already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/82ca6840-cf23-4bb0-a9be-55237ebbe9d3.


Let's build an agent that we can use for evaluation.

In [4]:
from langchain_benchmarks.tool_usage import agents

agent_factory = agents.OpenAIAgentFactory(task, model="gpt-3.5-turbo-16k")

# Let's test that our agent works
agent = agent_factory.create()
agent.invoke({"question": "abc"})

{'question': 'abc',
 'output': 'a, b, c',
 'intermediate_steps': [(AgentActionMessageLog(tool='type_letter', tool_input={'letter': 'a'}, log="\nInvoking: `type_letter` with `{'letter': 'a'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "letter": "a"\n}', 'name': 'type_letter'}})]),
   'OK'),
  (AgentActionMessageLog(tool='type_letter', tool_input={'letter': 'b'}, log="\nInvoking: `type_letter` with `{'letter': 'b'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "letter": "b"\n}', 'name': 'type_letter'}})]),
   'OK'),
  (AgentActionMessageLog(tool='type_letter', tool_input={'letter': 'c'}, log="\nInvoking: `type_letter` with `{'letter': 'c'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "letter": "c"\n}', 'name': 'type_letter'}})]),
   'OK')],
 'state': 'abc'}

## Eval

Let's evaluate an agent now

In [5]:
from langsmith.client import Client

from langchain_benchmarks.tool_usage import STANDARD_AGENT_EVALUATOR

client = Client()

test_run = client.run_on_dataset(
    dataset_name=task.name,
    llm_or_chain_factory=agent_factory.create,
    evaluation=STANDARD_AGENT_EVALUATOR,
    verbose=True,
    tags=["gpt-3.5-turbo-16k"],
)

View the evaluation results for project 'test-fresh-whip-11' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/c0c32118-d413-409f-ac01-088632c0e6ab?eval=true

View all tests for Dataset Tool Usage - Typewriter (1 tool) at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/82ca6840-cf23-4bb0-a9be-55237ebbe9d3
[------------------------------------------------->] 20/20
 Eval quantiles:
                                0.25  0.5  0.75  mean  mode
Intermediate steps correctness  1.00  1.0   1.0  0.95   1.0
# steps / # expected steps      1.00  1.0   1.0  1.70   1.0
Correct Final State             1.00  1.0   1.0  0.95   1.0
correctness                     0.75  1.0   1.0  0.75   1.0


# Inspect

You can take a look at the underlying results.

In [12]:
import pandas as pd

df = test_run.to_dataframe()
df = pd.json_normalize(df.to_dict(orient="records"))

In [13]:
df["correctness"].mean()

0.75

In [14]:
df["num_expected_steps"] = df["reference.expected_steps"].apply(len)
df["actual_number_of_steps"] = df["output.intermediate_steps"].apply(len)

In [15]:
df.head()

,Intermediate steps correctness,# steps / # expected steps,Correct Final State,correctness,input.question,output.question,output.output,output.intermediate_steps,output.state,reference.state,reference.reference,reference.expected_steps,num_expected_steps,actual_number_of_steps
0,1,1.0,1,1,communication,communication,communication,[(tool='type_letter' tool_input={'letter': 'c'...,communication,communication,communication,"[type_letter, type_letter, type_letter, type_l...",13,13
1,1,1.0,1,1,information,information,information,[(tool='type_letter' tool_input={'letter': 'i'...,information,information,information,"[type_letter, type_letter, type_letter, type_l...",11,11
2,1,1.0,1,1,dictionary,dictionary,dictionary,[(tool='type_letter' tool_input={'letter': 'd'...,dictionary,dictionary,dictionary,"[type_letter, type_letter, type_letter, type_l...",10,10
3,1,1.0,1,1,university,university,u\nn\ni\nv\ne\nr\ns\ni\nt\ny,[(tool='type_letter' tool_input={'letter': 'u'...,university,university,university,"[type_letter, type_letter, type_letter, type_l...",10,10
4,1,1.0,1,1,keyboard,keyboard,keyboard,[(tool='type_letter' tool_input={'letter': 'k'...,keyboard,keyboard,keyboard,"[type_letter, type_letter, type_letter, type_l...",8,8
